In [1]:
import cv2, pickle
import numpy as np
import tensorflow as tf
import os
import sqlite3
from keras.models import load_model

/home/rupamay/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.logging.set_verbosity(tf.logging.ERROR)
# classifier = tf.estimator.Estimator(model_dir="tmp/cnn_model2", model_fn=cnn_model_fn)
prediction = None
model = load_model('cnn_model_keras3.h5')

In [3]:
def get_image_size():
    img = cv2.imread('gestures/42/100.jpg', 0)
    return img.shape
    

In [4]:
image_x, image_y = get_image_size()
print(image_x,image_y)

50 50


In [5]:
def keras_process_image(img):
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (1, image_x, image_y, 1))
    return img

In [6]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [7]:
def get_pred_text_from_db(pred_class):
    conn = sqlite3.connect("gesture_db.db")
    cmd = "SELECT g_name FROM gesture WHERE g_id="+str(pred_class)
    cursor = conn.execute(cmd)
    for row in cursor:
        return row[0]

In [8]:
def split_sentence(text, num_of_words):
    '''
    Splits a text into group of num_of_words
    '''
    list_words = text.split(" ")
    length = len(list_words)
    splitted_sentence = []
    b_index = 0
    e_index = num_of_words
    while length > 0:
        part = ""
        for word in list_words[b_index:e_index]:
            part = part + " " + word
        splitted_sentence.append(part)
        b_index += num_of_words
        e_index += num_of_words
        length -= num_of_words
    return splitted_sentence

In [9]:
def put_splitted_text_in_blackboard(blackboard, splitted_text):
    y = 200
    for text in splitted_text:
        cv2.putText(blackboard, text, (4, y), cv2.FONT_HERSHEY_TRIPLEX, 2, (255, 255, 255))
        y += 50

In [10]:
def get_hand_hist():
    with open("hist3", "rb") as f:
        hist = pickle.load(f)
    return hist

In [11]:
def recognize():
    
    global prediction
    cam = cv2.VideoCapture(1)
    if cam.read()[0] == False:
        cam = cv2.VideoCapture(0)
    hist = get_hand_hist()
    x, y, w, h = 300, 100, 300, 300
    i=0
    # create SIFT feature extractor
    sift = cv2.xfeatures2d.SIFT_create()
    text = ""
    while True:
        
        text = ""
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        img = cv2.resize(img, (640, 480))
        imgCrop = img[y:y+h, x:x+w]
        imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([imgHSV], [0, 1], hist, [0, 180, 0, 256], 1)
        disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(16,16))
        cv2.filter2D(dst,-1,disc,dst)
        blur = cv2.bilateralFilter(dst, 9,100,75)
        blur = cv2.GaussianBlur(dst, (11,11), 0)
        blur = cv2.medianBlur(blur, 15)

        thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
        thresh = cv2.merge((thresh,thresh,thresh))
        thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        thresh = thresh[y:y+h, x:x+w]
        (openCV_ver,_,__) = cv2.__version__.split(".")
        if openCV_ver=='3':
            contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[1]
        elif openCV_ver=='4':
            contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
        if len(contours) > 0:
            contour = max(contours, key = cv2.contourArea)
            #print(cv2.contourArea(contour))
            if cv2.contourArea(contour) > 10000:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                save_img = thresh[y1:y1+h1, x1:x1+w1]

                if w1 > h1:
                    save_img = cv2.copyMakeBorder(save_img, int((w1-h1)/2) , int((w1-h1)/2) , 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1-w1)/2) , int((h1-w1)/2) , cv2.BORDER_CONSTANT, (0, 0, 0))
                if i >= 4:
                    save_img=cv2.equalizeHist(save_img)
#                     cv2.imshow("save_img", save_img)
#                     save_img = cv2.resize(save_img, (image_x, image_y))

                    pred_probab, pred_class = keras_predict(model, save_img)
                    path_url='gestures/'+ str(pred_class) +'/1.jpg'
                    img_gesture = cv2.imread(path_url,0)
                    original_keypoints, original_descriptor = sift.detectAndCompute( img_gesture, None)
                    query_keypoints, query_descriptor = sift.detectAndCompute(save_img,None)
                    brute_force = cv2.BFMatcher(cv2.NORM_L2, crossCheck = True)
                    matches = brute_force.match(original_descriptor, query_descriptor)
#                   matches = sorted(matches, key = lambda x : x.distance)
#                   result = cv2.drawMatches(img_gesture, original_keypoints, save_img, query_keypoints, matches, save_img, flags = 2)
#                   cv2.imshow("result",result)
#                   print("The number of matching keypoints between the original and the query image is {}\n".format(len(matches)))

                    if pred_probab*100 > 95 and len(matches)>=1:
                        text = get_pred_text_from_db(pred_class)
                        print(text)
                    i=0
                    
                    
                
#                     cv2.imshow("save_img", save_img)
        i+=1
        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
#         splitted_text = split_sentence(text, 2)
        
#         put_splitted_text_in_blackboard(blackboard, splitted_text)
        cv2.putText(blackboard, text, (30, 200), cv2.FONT_HERSHEY_TRIPLEX, 1.3, (255, 255, 255))
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        res = np.hstack((img, blackboard))
        
        
        cv2.imshow("Recognizing gesture", res)
        cv2.imshow("thresh", thresh)
        
        if cv2.waitKey(1) == ord('q'):
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            break

In [17]:

keras_predict(model, np.zeros((50, 50), dtype=np.uint8))
recognize()


1/1 [==============================] - 0s 13ms/step
E
1/1 [==============================] - 0s 13ms/step
H
1/1 [==============================] - 0s 15ms/step
H
1/1 [==============================] - 0s 13ms/step
I/Me 
1/1 [==============================] - 0s 13ms/step
I/Me 
1/1 [==============================] - 0s 13ms/step
No
1/1 [==============================] - 0s 13ms/step
L
1/1 [==============================] - 0s 13ms/step
H
1/1 [==============================] - 0s 13ms/step
H
1/1 [==============================] - 0s 13ms/step
H
1/1 [==============================] - 0s 13ms/step
H
1/1 [==============================] - 0s 13ms/step
Z
1/1 [==============================] - 0s 13ms/step
Z
1/1 [==============================] - 0s 13ms/step
Z
1/1 [==============================] - 0s 14ms/step
Best of Luck 
